In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [32]:
import numpy as np
from src.models.predict_model import KNN
from src.data.load_data import df
from src.features.build_features import Scaler
from scipy.stats import mode

In [4]:
df.head()

,Clump Thickness,Cell Size Uniformity,Cell Shape Uniformity,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nuclei,Mitoses,Class
Sample,,,,,,,,,,
1000025,5,1,1,1,2,1.0,3,1,1,2
1002945,5,4,4,5,7,10.0,3,2,1,2
1015425,3,1,1,1,2,2.0,3,1,1,2
1016277,6,8,8,1,3,4.0,3,7,1,2
1017023,4,1,1,3,2,1.0,3,1,1,2


In [5]:
x = df.values[:, :-1]
y = np.atleast_2d(df.values[:, -1]).T

In [6]:
x.shape

(683, 9)

In [7]:
y.shape

(683, 1)

In [12]:
knn = KNN(n=1)

In [13]:
knn.fit(x, y)

In [14]:
knn.predict(x[:5, :])

ModeResult(mode=array([[2.],
       [4.],
       [2.],
       [4.],
       [2.]]), count=array([[1],
       [1],
       [1],
       [1],
       [1]]))

In [15]:
y[:5]

array([[2.],
       [2.],
       [2.],
       [2.],
       [2.]])

In [16]:
from scipy.spatial.distance import cdist

In [17]:
cdist(x, x[:5])

array([[ 0.        , 11.87434209,  2.23606798, 12.04159458,  2.23606798],
       [11.87434209,  0.        , 11.3137085 , 11.22497216, 11.40175425],
       [ 2.23606798, 11.3137085 ,  0.        , 12.16552506,  2.44948974],
       ...,
       [17.3781472 , 14.6628783 , 17.40689519,  8.06225775, 17.29161647],
       [12.9614814 , 11.78982612, 12.76714533,  8.18535277, 12.60952021],
       [13.89244399, 10.95445115, 13.6381817 ,  8.94427191, 13.41640786]])

In [130]:
class KNN():
    def __init__(self, n, metric='euclidean'):
        self.n = n
        self.xs_ = None
        self.y_ = None
        self.scaler_ = None
        self.metric = metric
    
    def fit(self, x, y):
        self.scaler_ = Scaler()
        self.scaler_.fit(x)
        self.xs_ = self.scaler_.transform(x)
        self.y_ = y

    def predict(self, x):
        # verify training has happened
        if self.xs_ is None:
            raise ValueError("You must train the model first")
        
        # scale x
        xs = self.scaler_.transform(x)

        # calculate the distance from every training vector to every x
        # result will be n_training by n_predict columns
        dists = cdist(self.xs_, xs, self.metric)

        # sort those to find out the closest to each x_predict
        inds = np.argsort(dists, axis=0)[:self.n, :]
        inds = inds.T

        # match those indices to column predictions
        l = mode(self.y_[inds], axis=1)
        labels = l.mode[:, 0, :]

        return labels


In [142]:
knn = KNN(25)

In [143]:
knn.fit(x, y)

In [144]:
x[:2]

array([[ 5.,  1.,  1.,  1.,  2.,  1.,  3.,  1.,  1.],
       [ 5.,  4.,  4.,  5.,  7., 10.,  3.,  2.,  1.]])

In [145]:
knn.predict(x[:2])

array([[2.],
       [4.]])

In [146]:
y[:2]

array([[2.],
       [2.]])

In [147]:
t = np.random.rand(6).reshape(-1, 2)

In [57]:
t

array([[0.60202155, 0.46334895],
       [0.67244063, 0.19282396],
       [0.40778091, 0.19341963]])

In [58]:
np.argsort(t, axis=0)

array([[2, 1],
       [0, 2],
       [1, 0]])